# 1.1 隐式基类 - Object 
每个python 定义的类都会隐式的继承object 类 

In [1]:
class X:
    pass 
print(X.__class__)
print(X.__class__.__base__)

<class 'type'>
<class 'object'>


上述输出说明： 类其实是type类的一个对象， type类的基类为object 类

# 1.2 基类中的__init__()方法 
对象的生命周期主要包括创建、初始化、销毁   
继承自object类的子类，总是可以对它的属性进行拓展   
例如下面这个例子不需要对width和height 进行初始化

In [2]:
class Rectangle:
    def area(self):
        return self.width * self.height  # 这种方式在python中是合法的，但不建议使用

r = Rectangle()
r.width, r.height = 2, 3 # 在使用时赋值
r.area()

6

延迟初始化属性的设计虽然具有一定的灵活性，但从长远来看，这是一种糟糕的设计 

In [3]:
import this 

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


Explicit is better than implict 显示而非隐式，对于每个__init__都应当显示的指定要初始化的变量

### 1.3 基类中实现 __init__方法 

In [4]:
# 多态设计 
class Card:
    def __init__(self, rank, suit):
        """
        rank: 号码 
        suit: 花色
        """
        self.suit = suit 
        self.rank = rank 
        self.hard, self.soft = self._points() # 类内部函数命名以_开始投， 表示该函数可以被继承访问 

class NumberCard(Card):
    def _points(self):
        return int(self.rank), int(self.rank)

class FaceCard(Card):
    def _points(self):
        return 10, 10 

class AceCard(Card):
    def _points(self):
        return 1, 11


> 内部函数以 _ 开头命名，表示该函数可被继承访问    
> 以 __ 开头， 为类内私有函数命名， 该函数不可被继承访问  

- 常见的多态设计， 每个子类为_points 提供特有的实现 
- 所有的子类有相同的方法名和属性名 



In [5]:
# 如果只是简单的去定义牌，可以使用如下方式 
cards = [AceCard('A', '♠'), NumberCard('2', '♠')]

但这样的枚举，需要枚举52张牌，麻烦且容易出错，可以考虑工厂函数   
先看一些其他的方法 

### 1.4 使用__init__创建常量清单  

很多情况下，应用会包括一个常量集合， 静态常量 构成了 **策略(Strategy)** 或 **状态(State)** 的一部分  
python 中并没有提供简单而直接的方式来定义一个不可变对象。  

这个例子中，将花色定义为一个不可变对象是有意义的

In [6]:
class Suit:
    def __init__(self, name, symbol):
        """
        name: 花色名
        symbol: 花色标识
        """
        self.name = name 
        self.symbol = symbol 

Spade, Heart, Club, Diamond = Suit('Spade', '♠'), Suit('Heart', '♥'), Suit('Club', '♣'), Suit('Diamond', '♦')

把创建好的花色对象做缓存，使得在调用时对象可以被重复使用，性能将会得到显著提升 
(python 对象只是概念上的常量，事实上仍然是可变的， 使用额外的代码使得对象完全不可变可能会更好)

### 1.5 通过工厂函数调用__init__ 

实现工厂有两种途径 
- 1. 定义一个函数， 返回不同的类 
- 2. 定义一个类， 包含了创建对象的方法  

第二种方式是完整的工厂设计模式，在类似java的语言中，工厂类层次结构是必须的， 因为*语言本身不支持脱离类而单独存在的函数* (好像还真是)  
在 python 里 类不是必须的， 只有在特别复杂的场景下，工厂类才是不错的选择  
python 的优势之一就是 对于只需要简单定义一个函数就能做到的事没有必要去定义类层次结构  

类定义的优势是 可以通过继承来使得代码被更好的重用  （而不仅仅是封装起来比较好看而已 ）

如果工厂类并没有重用任何代码，那么类层次结构在python中并没有多大用处， 完全可以用函数替代  

如下是一个工厂函数的例子

In [7]:
def card(rank, suit):
    """ 生成对应的card 对象的工厂函数 
    rank: 号码 
    suit: 花色
    """
    if rank == 1: 
        return AceCard('A', suit) 
    elif 2 <= rank <= 10:
        return NumberCard(str(rank), suit) 
    elif 11<= rank <= 13:
        name = {
            11: 'J',
            12: 'Q',
            13: 'K'
        }
        return FaceCard(name[rank], suit)
    else: # 应当尽量避免不明确的分支， 枚举所有场景后通过 else throw exception 是一个更明智的方法 
        raise Exception('Rank out of range')
    
# 可以通过这种方式来构建卡片对象 
deck = [card(rank, suit) for rank in range(1, 14) for suit in (Spade, Heart, Club, Diamond)]

In [8]:
## version 2  使用映射来简化设计 
def card4(rank, suit):
    """
    使用映射来简化上述工厂方法  
    """
    return {
        1: AceCard('A', suit),
        11: FaceCard('J', suit),
        12: FaceCard('Q', suit),
        13: FaceCard('K', suit)
    }.get(rank, NumberCard(str(rank), suit))

### v3: 代码重复度进一步减少 
def card5(rank, suit):
    """  映射到一个二元组
    """
    class_, rank_str = {
        1: (AceCard, 'A'),  # 类本身也是一个对象 
        11: (FaceCard, 'J'),
        12: (FaceCard, 'Q'),
        13: (FaceCard, 'K')
    }.get(rank, (NumberCard, str(rank)))
    return class_(rank_str, suit)

deck = [card4(rank, suit) for rank in range(1, 14) for suit in (Spade, Heart, Club, Diamond)]

上述card5 实现由两个缺点 
- 从rank 值 映射到 类对象 很少见   
- 两个参数只有一个用于类的初始化 

从rank 映射到一个相对简单的类或函数对象，而不必提供目的不明确的参数，是一个更好的选择

In [9]:
from functools import partial 
def card6(rank, suit):
    """ 使用partial 函数来进行类对象的生产 
    """
    class_ = {
        1: partial(AceCard, 'A'),
        11: partial(FaceCard, 'J'),
        12: partial(FaceCard, 'Q'),
        13: partial(FaceCard, 'K')
    }.get(rank, partial(NumberCard, str(rank)))
    return class_(suit)

# 重写花色类方法的生成 
def suit2(symbol):
    """ 花色类重写 """
    class_ = {
        '♠': partial(Suit, 'Spade'),
        '♥': partial(Suit, 'Heart'),
        '♣': partial(Suit, 'Club'),
        '♦': partial(Suit, 'Diamond')
    }.get(symbol)

    return class_(symbol)


partial() 函数的使用在函数式编程中很常见 

### 1.6 在每个子类中实现__init__()方法  
考虑重构card类，并将部分初始化在card类中实现 

In [10]:
class Card:
    def __init__(self, rank, suit, hard, soft):
        """
        rank: 号码 
        suit: 花色 
        hard: 
        soft
        """
        self.rank = rank 
        self.suit = suit 
        self.hard = hard 
        self.soft = soft 

class AceCard(Card):
    def __init__(self, rank, suit):
        super().__init__('A', suit, 1, 11)
    
class NumberCard(Card):
    def __init__(self, rank, suit):
        super().__init__(str(rank), suit, int(rank), int(rank))

class FaceCard(Card):
    def __init__(self, rank, suit):
        super().__init__({11: 'J', 12: 'Q', 13: 'K'}.get(int(rank)), suit, 10, 10)

经过上述重写以后，工厂函数的实现也会变得简洁 
> 事实上， 在__init__ 函数和 工厂函数之间有些权衡，通常直接调用 比 __init___并把构造复杂性分发给工厂函数 更好， 当需要封装复杂的构造逻辑时，工厂函数才是一个更好的选择 

In [11]:
def card10(rank, suit):
    """ 工厂方法v10 """
    if rank == 1: return AceCard(rank, suit) 
    elif 2 <= rank < 11: return NumberCard(rank, suit) 
    elif 11 <= rank < 14: return FaceCard(rank, suit) 
    else: raise Exception('rank is invalid ')
    

### 1.7 简单的组合对象 
一个组合对象也可以被称为**容器** 
> 面向对象的设计原则 
> 单一功能原则 : 对象应该仅具有一种单一功能   
> 开闭原则： 对于拓展开发，对于修改封闭    
> 里氏替换原则： 程序中的对象应该是在可以不改变程序正确性的前提下被它的子类所替换   
> 接口隔离原则： 多个特定的客户端接口要好于一个用途宽泛的接口    
> 依赖反转原则： 一个方法应该遵守依赖于抽象而不是一个实例原则    

类定义的一个优势： 类给对象提供了简单的、不需要实现的接口 （然而对于python来说，却不是必须的，可能这就是脚本语言的特性吧）

设计一个集合类，通常有一下三种方法：  
- 封装： 这个设计是基于现有集合类来定义一个新类，属于外观模式的一个使用场景 
- 扩展： 对现有集合类进行拓展， 通常使用定义子类的方式来实现  
- 创建： 重新设计 

In [15]:
"""
如下是一个封装的例子， 内部事实上是对 list 的一个封装， 调用方法也都是list 的方法
"""
import random 
class Deck:
    def __init__(self):
        # 牌堆
        self._cards = [card10(rank, suit) for rank in range(1, 14) for suit in [Spade, Heart, Club, Diamond]]
        random.shuffle(self._cards) 

    def pop(self):
        """ 发牌 """
        return self._cards.pop()
    

一般来说， 外观模式或者封装类中的方法实现只是对底层对象相应函数的代理调用（有点多余?）

In [16]:
"""
拓展集合类, 好处是不用重新实现Pop方法, 只需继承即可
"""
class Deck(list):
    def __init__(self):
        _cards = [card10(rank, suit) for rank in range(1, 14) for suit in [Spade, Heart, Club, Diamond]]
        super().__init__(_cards)
        random.shuffle(self) #  牛啊？self 还可以这么用 ？ 

In [19]:
deck = Deck()
print(deck[0].rank)

Q
